In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os
import glob
import pandas as pd
import numpy as np
from scipy import interpolate
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from google.colab import drive
import pickle
from scipy.io import wavfile
from scipy.signal import decimate
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, concatenate, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [ ]:
### Loading presaved datasets
import pickle

with open('local_train_x_w.pkl', 'rb') as file:
    local_train_x_w = pickle.load(file)
with open('local_train_x_p.pkl', 'rb') as file:
    local_train_x_p = pickle.load(file)
with open('local_train_x_s.pkl', 'rb') as file:
    local_train_x_s = pickle.load(file)
with open('local_train_y.pkl', 'rb') as file:
    local_train_y = pickle.load(file)

with open('local_test_x_w.pkl', 'rb') as file:
    local_test_x_w = pickle.load(file)
with open('local_test_x_p.pkl', 'rb') as file:
    local_test_x_p = pickle.load(file)
with open('local_test_x_s.pkl', 'rb') as file:
    local_test_x_s = pickle.load(file)
with open('local_test_y.pkl', 'rb') as file:
    local_test_y = pickle.load(file)


with open('central_test_x_w.pkl', 'rb') as file:
    central_test_x_w = pickle.load(file)
with open('central_test_x_p.pkl', 'rb') as file:
    central_test_x_p = pickle.load(file)
with open('central_test_x_s.pkl', 'rb') as file:
    central_test_x_s = pickle.load(file)
with open('central_test_y.pkl', 'rb') as file:
    central_test_y = pickle.load(file)


with open('central_train_x_w.pkl', 'rb') as file:
    central_train_x_w = pickle.load(file)
with open('central_train_x_p.pkl', 'rb') as file:
    central_train_x_p = pickle.load(file)
with open('central_train_x_s.pkl', 'rb') as file:
    central_train_x_s = pickle.load(file)
with open('central_train_y.pkl', 'rb') as file:
    central_train_y = pickle.load(file)


with open('ood_watch_test.pkl', 'rb') as file:
    ood_test_x_w = pickle.load(file)
with open('ood_phone_test.pkl', 'rb') as file:
    ood_test_x_p = pickle.load(file)
with open('ood_audio_test.pkl', 'rb') as file:
    ood_test_x_s = pickle.load(file)
with open('ood_label_test.pkl', 'rb') as file:
    ood_test_y = pickle.load(file)

# Processing data

In [ ]:
for client in range(28):
  local_paired_watch[client]=np.reshape(local_paired_watch[client],(-1,100,6,1))
  local_paired_phone[client]=np.reshape(local_paired_phone[client],(-1,100,21,1))
  local_paired_audio[client]=np.reshape(local_paired_audio[client],(-1,25,32,1))
  local_paired_label[client] = np.array(local_paired_label[client])

# normalizing local datasets
for i in range(28):
    shape_w = local_paired_watch[i].shape
    shape_p = local_paired_phone[i].shape
    shape_s = local_paired_audio[i].shape
    x_2d_w = local_paired_watch[i].reshape(shape_w[0], -1)
    x_2d_p = local_paired_phone[i].reshape(shape_p[0], -1)
    x_2d_s = local_paired_audio[i].reshape(shape_s[0], -1)
    # Initialize a scaler
    scaler = StandardScaler()
    x_normalized_2d = scaler.fit_transform(x_2d_w)
    local_paired_watch[i] = x_normalized_2d.reshape(shape_w)

    scaler = StandardScaler()
    x_normalized_2d = scaler.fit_transform(x_2d_p)
    local_paired_phone[i] = x_normalized_2d.reshape(shape_p)

    scaler = StandardScaler()
    x_normalized_2d = scaler.fit_transform(x_2d_s)
    local_paired_audio[i] = x_normalized_2d.reshape(shape_s)

del x_2d_p, x_2d_s,x_2d_w, x_normalized_2d

In [ ]:
##### Data for Federated Learning

local_paired_watch_train = list(); local_paired_watch_test = list();
local_paired_phone_train = list(); local_paired_phone_test = list();
local_paired_audio_train = list(); local_paired_audio_test = list();
local_paired_label_train = list(); local_paired_label_test = list();

for i in range(20):
    x_train_watch, x_test_watch,x_train_phone, x_test_phone,x_train_audio, x_test_audio, y_train, y_test =\
    train_test_split(local_paired_watch[i],local_paired_phone[i],local_paired_audio[i],local_paired_label[i],test_size=0.25,\
                     random_state=42)

    local_paired_watch_train.append(x_train_watch)
    local_paired_phone_train.append(x_train_phone)
    local_paired_audio_train.append(x_train_audio)
    local_paired_label_train.append(y_train)

    local_paired_watch_test.append(x_test_watch)
    local_paired_phone_test.append(x_test_phone)
    local_paired_audio_test.append(x_test_audio)
    local_paired_label_test.append(y_test)

    if i == 0:
        global_paired_watch_test = x_test_watch
        global_paired_phone_test = x_test_phone
        global_paired_audio_test = x_test_audio
        global_paired_label_test = y_test
    else:
        global_paired_watch_test = np.concatenate([global_paired_watch_test,x_test_watch],axis=0)
        global_paired_phone_test = np.concatenate([global_paired_phone_test,x_test_phone],axis=0)
        global_paired_audio_test = np.concatenate([global_paired_audio_test,x_test_audio],axis=0)
        global_paired_label_test = np.concatenate([global_paired_label_test,y_test],axis=0)

In [ ]:
## Data for Centralized Learning

for i in range(20):
    if i == 0:
        central_paired_watch_train = local_paired_watch_train[i]
        central_paired_phone_train = local_paired_phone_train[i]
        central_paired_audio_train = local_paired_audio_train[i]
        central_paired_label_train = local_paired_label_train[i]
    else:
        central_paired_watch_train = np.concatenate((central_paired_watch_train,local_paired_watch_train[i]),axis=0)
        central_paired_phone_train = np.concatenate((central_paired_phone_train,local_paired_phone_train[i]),axis=0)
        central_paired_audio_train = np.concatenate((central_paired_audio_train,local_paired_audio_train[i]),axis=0)
        central_paired_label_train = np.concatenate((central_paired_label_train,local_paired_label_train[i]),axis=0)

for i in range(20,28,1):
  if i == 20:
    ood_watch_test = local_paired_watch[i]
    ood_phone_test = local_paired_phone[i]
    ood_audio_test = local_paired_audio[i]
    ood_label_test = local_paired_label[i]
  else:
    ood_watch_test = np.concatenate((ood_watch_test,local_paired_watch[i]),axis=0)
    ood_phone_test = np.concatenate((ood_phone_test,local_paired_phone[i]),axis=0)
    ood_audio_test = np.concatenate((ood_audio_test,local_paired_audio[i]),axis=0)
    ood_label_test = np.concatenate((ood_label_test,local_paired_label[i]),axis=0)